In [1]:
from LightningModules import AutoEncoder, FUCCIDataModule, ReconstructionVisualization
from kornia import tensor_to_image
from microfilm.colorify import multichannel_to_rgb
import numpy as np
from PIL import Image
from pathlib import Path

In [6]:
FUCCI_PATH = '/home/ishang/cross-modal-autoencoders/FUCCI-dev-data'
LIGHTNING_LOGS = '/data/ishang/fucci_vae/lightning_logs'
!ls {LIGHTNING_LOGS}

FUCCI_reference_VAE		      FUCCI_reference_VAE_28_07_05_14_2023
FUCCI_reference_VAE_14_05_05_14_2023  FUCCI_reference_VAE_33_04_05_14_2023
FUCCI_reference_VAE_2023_16_05_04_37  FUCCI_reference_VAE_54_07_05_13_2023


In [9]:
model_folder = 'FUCCI_reference_VAE_2023_16_05_04_37'
!ls {LIGHTNING_LOGS}/{model_folder}
checkpoint = 'epoch=02-Val_loss=0.00.ckpt'
model_path = Path(LIGHTNING_LOGS) / model_folder / checkpoint
res = 1024

'epoch=02-Val_loss=0.00.ckpt'


In [13]:
import torch
ckpt = torch.load(model_path)
encoder = ckpt['state_dict']['encoder']
decoder = ckpt['decoder']
model = AutoEncoder(encoder, decoder)

KeyError: 'encoder'

In [11]:
model = AutoEncoder.load_from_checkpoint(model_path)

TypeError: AutoEncoder.__init__() missing 2 required positional arguments: 'encoder' and 'decoder'

In [10]:
dm = FUCCIDataModule(
    data_dir=FUCCI_PATH,
    dataset="reference",
    imsize=res,
    split=(0.8, 0.1, 0.1),
    batch_size=8,
    num_workers=16
)

In [ ]:
print(dm.dataset[:3].shape)
print(dm.dataset[0].shape)

torch.Size([3, 2, 1024, 1024])
torch.Size([2, 1024, 1024])


In [ ]:
grid = ReconstructionVisualization.make_reconstruction_grid(dm.dataset[:3], dm.dataset[:3])
print(grid.shape)
img = np.moveaxis(tensor_to_image(grid), -1, 0)
print(img.shape)
image_composite, _, _, _= multichannel_to_rgb(img, cmaps=dm.dataset.channel_colors())
image_composite = (255 * image_composite).astype(np.uint8)
image_composite = Image.fromarray(image_composite)
image_composite.save(f'test_grid_{res}_composite.png')

torch.Size([2, 3080, 2054])
(2, 3080, 2054)
